In [6]:
from pathlib import Path
import logging

from src.anki import read_file, make_deck, make_model, make_notes, make_package, read_package
from src.media import add_images, add_phonetics, add_sounds
from src.log_config import configure_logging

configure_logging(
    stream_level=logging.DEBUG, 
    ignore_libs=[
        "PIL", "phonemizer", "urllib3", 
        "httpx", "httpcore","gtts", "openai"])

INPUT_DIR = Path("input/")
OUTPUT_DIR = Path("output/")
TEMPLATE_DIR = Path("template/")
MEDIA_DIR = Path("media/")
SOUND_DIR = MEDIA_DIR / "sound"
IMG_DIR = MEDIA_DIR / "img"
for d in [INPUT_DIR, OUTPUT_DIR, TEMPLATE_DIR, MEDIA_DIR, SOUND_DIR, IMG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

language = "fr"

top_deck_name = "Français"
file_name = "Les_nombres"
# file_name = "L'alphabet"
file_name = "Cours_01"
# file_name = "Cours_01_Q&A"
file_name = "Cours_02"
# file_name = "Cours_03"
# file_name = "Cours_03_Q&A"
# file_name = "Cours_04"

deck_name = top_deck_name + "::" + file_name if top_deck_name else file_name
file_type = ".md"
file_path = INPUT_DIR / (file_name + file_type)
apkg_path = OUTPUT_DIR / (file_name + ".apkg")
style = TEMPLATE_DIR / "style.css"
img_paths, sound_paths = [],[]

In [7]:
df = read_file(file_path)
df = add_phonetics(df, language)
df, sound_paths = add_sounds(df, SOUND_DIR, language, engine="gtts", force_replace=False)
df, img_paths = add_images(df, IMG_DIR, language, engine="bing", force_replace=False)
media_paths = sound_paths + img_paths

deck = make_deck(df, deck_name)
model = make_model(df, style)
make_notes(df, deck, model)
make_package(deck, media_paths=media_paths, apkg_path=apkg_path)
df.head()

[22:47:44] [INFO] [media] - Added phonetics for 53 words.
[22:47:44] [DEBUG] [media] - Sound for 'le jour' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'la semaine' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'la semaine dernière' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'fumer' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'un jeu de chance' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'un jour férié' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'parler' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'Je parle un peu allemand.' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'Je parle un peu français.' already exists, skipping replacement.
[22:47:44] [DEBUG] [media] - Sound for 'allemand' already exists, skipping replacement.
[22:47:4

,Front,Back,Remark,More,Phonetics,Image,Sound,Reverse
0,le jour,der Tag,,,/lə- ʒˈuʁ/,"<img src=""img_badffde5fef4137a.png"">",[sound:sound_badffde5fef4137a.mp3],
1,la semaine,die Woche,,,/la- səmˈɛn/,"<img src=""img_1387874aa21491e5.png"">",[sound:sound_1387874aa21491e5.mp3],
2,la semaine dernière,letzte Woche,,,/la- səmˈɛn dɛʁnjˈɛʁ/,"<img src=""img_ff13f511caf0e865.png"">",[sound:sound_ff13f511caf0e865.mp3],
3,fumer,rauchen,,,/fymˈe/,"<img src=""img_3349713c4db68bab.png"">",[sound:sound_3349713c4db68bab.mp3],
4,un jeu de chance,ein Glückspiel,,,/œ̃ ʒˈø də- ʃˈɑ̃s/,"<img src=""img_bb87177e976487c8.png"">",[sound:sound_bb87177e976487c8.mp3],


In [8]:
import os
from pathlib import Path
from phonemizer.backend import EspeakBackend
from phonemizer.backend.espeak.wrapper import EspeakWrapper

if os.uname().sysname == 'Darwin':
    EspeakWrapper.set_library(Path("/opt/local/bin/espeak")) # macports version

language = "fr-fr"
backend = EspeakBackend(language=language, with_stress=True, preserve_punctuation=True)
phonemize = lambda x: f"{backend.phonemize([str(x)], strip=True)[0]}"


In [11]:
word = """
un policier
"""

phonemize(word)

'œ̃ polisjˈe'

In [10]:
import openai


def chatgpt(prompt, model="davinci"):
    """
    Generates a chatbot response from a prompt using OpenAI's API.
    """
    client = openai.OpenAI()
    try:
        response = client.Completion.create(
            engine=model,
            prompt=f"{prompt}\n\nUser:",
            temperature=0.9,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0.6,
            stop=["\n", " User:"]
        )
        return response.choices[0].text.strip()
    except openai.OpenAIError as e:
        print(e)
        return None
    